## <CENTER> The $Z^0$ boson as an example
_____________________________________________

### Using what you have learned so far, let's see if we can find the $Z^0$ boson as an example!
_____________________________________________

Install ROOT if working in Google Colab (if using this notebook via our docker image it's already there).

In [ ]:
# install the ROOT libraries if you're using Google Colab - this usually takes a minute or two
# if not we assume we're running in the docker container where it is already available
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    # install ROOT and update a few other packages
    !wget https://github.com/MohamedElashri/ROOT/releases/download/ubuntu/root_v6.28.04_Ubuntu_20.04.zip
    !unzip /content/root_v6.28.04_Ubuntu_20.04.zip
    !apt-get install git dpkg-dev cmake g++ gcc binutils libx11-dev libxpm-dev libxft-dev libxext-dev tar gfortran subversion
    #!apt-get install libpython3.6-dev # seems to not be needed
    # the following is needed because colab upgraded the openssl library
    !wget http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb
    !sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2_amd64.deb
    # add some paths etc so that libraries can be found
    import sys
    sys.path.append("/content/root_build/")
    sys.path.append("/content/root_build/bin/")
    sys.path.append("/content/root_build/include/")
    sys.path.append("/content/root_build/lib/")
    import ctypes
    ctypes.cdll.LoadLibrary('/content/root_build/lib//libCore.so')
    ctypes.cdll.LoadLibrary('/content/root_build/lib//libThread.so')
    ctypes.cdll.LoadLibrary('/content/root_build/lib//libTreePlayer.so')
    print("Done installing ROOT - check that there were no error messages above!")
else:
    print("Not running the notebook in Google Colab, so assuming ROOT is already available")

In [ ]:
# import the ROOT library
import ROOT

### Get the data

There are several data sets from ATLAS that are publicly available, and you can find the latest batch from 2020 at this URL: https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/

Clicking around on the above page, you will find simulated data and real experimental data, in divided into categories based on the final states seen by the ATLAS detector. For this exercise we want to study the Z boson decaying to charged leptons, so we'll look at so-called dilepton events. Navigate into the "2lep" directory, then "data". You will find several files - let's see how we can access these from our code!


In [ ]:
# we can load all the files in sequence in a "chain"
# the tree we want to access inside the files is called "mini"
chains = {}
chains["data"] = ROOT.TChain("mini")

# there are several files for the data, add the paths to the list below to analyze more
for file in ["https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/Data/data_A.2lep.root"]:
    # the file needs to be downloaded if you're in Google Colab, which will take a while depending on its size
    if IN_COLAB:
        !wget $file # download the file locally
        print(file)
        !ls -l
        file = file.split('/')[-1] # add the file without the path since it's now local
    chains["data"].Add(file)
    
# let's also look at some simulated data - there are lots of processes that can give two leptons 
# listed in the MC dir, but we'll only use Z(ee) and Z(mumu) for now
chains["mc2e"] = ROOT.TChain("mini")
file = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_361106.Zee.2lep.root"
if IN_COLAB:
    !wget $file
    file = file.split('/')[-1]
chains["mc2e"].Add(file)

chains["mc2mu"] = ROOT.TChain("mini")
file = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_361107.Zmumu.2lep.root"
file = $file
if IN_COLAB:
    !wget $file
    file = file.split('/')[-1]
chains["mc2mu"].Add(file)

for s in chains:
    print("Loaded sample %s containing %d events" % (s, chains[s].GetEntries()))

### Helper function for invariant mass

In [ ]:
# this function calculates the invariant mass of a two-particle system
def getInvMass(lep1_pt, lep1_eta, lep1_phi, lep2_pt, lep2_eta, lep2_phi):
    ''' Compute the invariant mass using the formula from the Introduction notebook (assume massless particles)'''
    import math
    m2 = 2*lep1_pt*lep2_pt*(math.cosh(lep1_eta-lep2_eta)-math.cos(lep1_phi-lep2_phi))
    return math.sqrt(m2)

### Prepare histograms

In [ ]:
# create a histogram for invariant mass

# define the number of bins and axis range
nBins = 150
lowerEdge = 0
upperEdge = 150

# make a python dictionary holding the histograms for each of the samples
massHistos = {}

# loop over the samples to create the histograms and the counters
for sample in chains:
    massHistos[sample+"_ee"] = ROOT.TH1F("InvariantMass_%s_ee" % sample, "; Invariant mass [GeV]; Number of events", nBins, lowerEdge, upperEdge)
    massHistos[sample+"_mumu"] = ROOT.TH1F("InvariantMass_%s_mumu" % sample, "; Invariant mass [GeV]; Number of events", nBins, lowerEdge, upperEdge)
    


### Run the event loop, find electron-positron/muon-antimuon pairs, evaluate the invariant mass, fill histogram(s). You are looking at data-not all events will contain a $Z^0$ boson. What cuts/requirements do you need to apply to filter out $Z^0$ boson candidate events? In the notebook 1 the different variables in the tree are listed.

In [ ]:
# clear the histogram, so you don't fill the same events several times if you re-run the loop
for s in massHistos:
    massHistos[s].Reset()

# loop over all of the samples, i.e. both data and the two simulated MC samples
for s in chains:
    # keep track of the number of events processed
    nEvents = 0
    maxEvents = 10000 # stop after this many events, useful during development
    # also keep track of the number of electron-positron and muon-antimuon pairs found
    for event in chains[s]:
        if nEvents > maxEvents:
            break
        # only process events which have exactly two leptons in them, as we expect for Z->ll
        if event.lep_n == 2:
            # skip events with different-flavor lepton pairs
            if event.lep_type[0] != event.lep_type[1]:
                continue
            # also skip events with same-sign leptons, we don't expect that from Z decays
            if event.lep_charge[0] == event.lep_charge[1]:
                continue
            m = getInvMass(event.lep_pt[0], event.lep_eta[0], event.lep_phi[0],
                          event.lep_pt[1], event.lep_eta[1], event.lep_phi[1])
            # fill the right histogram depending on the type of lepton pair found
            if event.lep_type[0] == 11:
                massHistos[s+"_ee"].Fill(m/1000) # divide by 1000 to go from MeV to GeV
            elif event.lep_type[0] == 13:
                massHistos[s+"_mumu"].Fill(m/1000) # divide by 1000 to go from MeV to GeV
            if nEvents % 1000 == 0: # only do a print-out every 1000 events
                print("Processed event %d of %s, filled m = %f" % (nEvents, s, m))
        nEvents += 1

**Note:**: Since it takes a long time to run over all data, only run over a small number of events while developing. Then, when you are satisfied with the method, you can increase the number of events.

### Draw the invariant mass distribution - do you see the $Z^0$ boson peak?

Let's first have a look at the two MC samples, to see if there are any significant differences expected between electrons and muons

In [ ]:
# let's look at the ee case first
eeDataCanvas = ROOT.TCanvas("Invariant mass ee", "Invariant mass ee", 800, 600)

massHistos["data_ee"].SetMarkerStyle(20)
massHistos["data_ee"].SetLineColor(1)
massHistos["data_ee"].Draw() # draw once so that the histogram has axes
# then set the y-axis range
yMax = 1.2*massHistos["data_ee"].GetBinContent(massHistos["data_ee"].GetMaximumBin())
massHistos["data_ee"].GetYaxis().SetRangeUser(0, yMax)
massHistos["data_ee"].Draw("E") # "E" draws data points with simple error bars

eeDataCanvas.Draw()

In [ ]:
# now let's try to fit a Breit-Wigner curve to the measured invariant mass distribution

# let's define a B-W function with a normalization parameter, and set reasonable starting values
bw = ROOT.TF1("myBreitWigner", "[norm]*TMath::BreitWigner(x, [mean], [gamma])", 80, 100)
bw.SetParameter("norm", 100)
bw.SetParameter("mean", 90)
bw.SetParameter("gamma", 3)

# do the fit
fitResult = massHistos["data_ee"].Fit(bw, "L S", "", 80, 100)
print("\n*** Chi^2 sum = {:.1f}, N_DoF = {}, ratio = {:.1f}".format(fitResult.Chi2(),
                                                                    fitResult.Ndf(),
                                                                    fitResult.Chi2()/fitResult.Ndf()))

# draw the canvas again to see the fit
eeDataCanvas.Draw()

### How good is the fit? Can you improve the it?
Does the measured mass come out near the true value? What does the $\chi^2/N_{DoF}$ say about the fit quality? 

**If you want to get better at the craft of fitting:**
* What can you do to improve the fit? 
* Could adding a (small) additional function that describes the background help? 
* Could something be done to account for the effects from imperfect detector/measurements?

In [ ]:
# now let's draw both data and MC so that we can see how well the simulations
# describe the data

# let's look at the ee case first
eeCanvas = ROOT.TCanvas("Invariant mass ee data-MC", "Invariant mass ee data-MC", 800, 600)

yMax = 1.2*massHistos["data_ee"].GetBinContent(massHistos["data_ee"].GetMaximumBin())
massHistos["data_ee"].GetYaxis().SetRangeUser(0, yMax)
massHistos["data_ee"].Draw("E") # "E" draws data points with simple error bars

# scale the MC histogram to the data to more easily compare the shapes
# use the integral around the peak to normalize
maxBinMC = massHistos["mc2e_ee"].GetMaximumBin()
integralMC = massHistos["mc2e_ee"].Integral(maxBinMC-3, maxBinMC+3)
maxBinData = massHistos["data_ee"].GetMaximumBin()
integralData = massHistos["data_ee"].Integral(maxBinData-3, maxBinData+3)
massHistos["mc2e_ee"].Scale(integralData/integralMC)

# set colors to see data, fit and MC more easily, see https://root.cern.ch/doc/master/classTAttFill.html for options
massHistos["mc2e_ee"].SetLineColor(ROOT.kAzure-9)
massHistos["mc2e_ee"].SetFillColor(ROOT.kAzure-9)

massHistos["mc2e_ee"].Draw("HIST SAME")
massHistos["data_ee"].Draw("SAME E") # draw the data again so that it's on top

eeCanvas.Draw()

In [ ]:
# now do the same for the muon-antimuon pairs!

...

### You're now ready to go hunting for new particles yourself in [4-Explore](4-Explore.ipynb)!